### 5.7: News Classifications

##### Objective

This workshop aims to explore Recurrent Neural Networks (RNNs) and Long Short-Term Memory (LSTM) architectures for text-based tasks. We will learn how to preprocess text data (through tokenisation and word embedding), apply RNNs to a news classification problem, and use LSTMs for text generation tasks such as poetry creation.

##### Learning Outcomes

1. **Text Tokenization and Embedding:** Learn how to transform raw text into a suitable input format (tokens) and represent words or tokens as dense vector embeddings for neural network processing.
2. **RNN-Based Classification:** The AG News dataset exemplifies applying a simple RNN model to a text classification task.
3. **LSTM for Text Generation:** Gain hands-on experience building an LSTM model to generate text using the Adele.txt poetry dataset as a case study.

##### Introduction to Text Modelling for Neural Networks

When dealing with textual data, we must convert language into numerical representations that neural networks can process. This involves two key steps:

1. **Tokenisation:** Splitting the text into smaller units, usually words (or sometimes sub-words or characters).
2. **Word Embedding:** Mapping each token to a continuous vector space (e.g., via word2vec or Glove-like embeddings) so semantically similar tokens have similar vector representations.

These representations capture syntactic and semantic relationships, enabling models to learn patterns across sequences of words.


##### Recurrent Neural Networks (RNN) and LSTM

`Recurrent Neural Networks (RNNs)` are designed to handle sequential data by maintaining a hidden state that propagates information from one time step to the next. Traditional RNNs, however, often struggle with long-term dependencies due to vanishing or exploding gradients.

`Long-short-term memory (LSTM)` networks address these issues by introducing a memory cell and gating mechanisms (input, output, and forget gates) that regulate how information flows through the network. This allows the model to maintain longer-range dependencies, making it especially effective for language modelling and text generation tasks.


##### Example Applications

1. **AG News Classification:**
    - Use tokenized AG News articles and feed them into an RNN architecture for topic classification.
    - The model learns to identify news categories (e.g., World, Business, Sports) by capturing patterns in word usage.
2. **LSTM for Poetry Generation:**
    - Train an LSTM on the Adele.txt dataset (or any poetry corpus) to learn linguistic styles and generate new verses.
    - By sampling from the trained model, you can produce novel lines of text that mimic the style of the training data.

##### Student Task

- Select a book dataset of your choice and preprocess the text for training.
- Implement a bidirectional LSTM or GRU model using TensorFlow for text generation.
- Train the model to learn the text style and structure, then generate new text samples.
- Share your code, model configuration, training strategies, and generated samples in the collaborative coding discussion forum.

##### Conclusion

In this workshop, you explored how tokenisation, word embedding, and recurrent architectures (RNN, LSTM) form the backbone of many Natural Language Processing (NLP) applications. By implementing classification and text-generation tasks, you gain practical insights into how neural networks handle sequential data. These skills will be a solid foundation for more advanced NLP techniques, such as Transformer-based models and attention mechanisms. 

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# Bidirectional LSTM/GRU for Text Generation

# Extended sonnets text (first 20 for demo; use full 154 for production)
sonnets = """
THE SONNETS

by William Shakespeare

I

From fairest creatures we desire increase,
That thereby beauty’s rose might never die,
But as the riper should by time decease,
His tender heir might bear his memory:
But thou, contracted to thine own bright eyes,
Feed’st thy light’s flame with self-substantial fuel,
Making a famine where abundance lies,
Thyself thy foe, to thy sweet self too cruel:
Thou that art now the world’s fresh ornament,
And only herald to the gaudy spring,
Within thine own bud buriest thy content,
And tender churl mak’st waste in niggarding:
    Pity the world, or else this glutton be,
    To eat the world’s due, by the grave and thee.

II

When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty’s field,
Thy youth’s proud livery so gazed on now,
Will be a tatter’d weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv’d thy beauty’s use,
If thou couldst answer ‘This fair child of mine
Shall sum my count, and make my old excuse,’
Proving his beauty by succession thine!
    This were to be new made when thou art old,
    And see thy blood warm when thou feel’st it cold.

III

Look in thy glass and tell the face thou viewest
Now is the time that face should form another;
Whose fresh repair if now thou not renewest,
Thou dost beguile the world, unbless some mother.
For where is she so fair whose unear’d womb
Disdains the tillage of thy husbandry?
Or who is he so fond will be the tomb,
Of his self-love to stop posterity?
Thou art thy mother’s glass and she in thee
Calls back the lovely April of her prime;
So thou through windows of thine age shalt see,
Despite of wrinkles this thy golden time.
    But if thou live, remember’d not to be,
    Die single and thine image dies with thee.

IV

Unthrifty loveliness, why dost thou spend
Upon thyself thy beauty’s legacy?
Nature’s bequest gives nothing, but doth lend,
And being frank she lends to those are free:
Then, beauteous niggard, why dost thou abuse
The bounteous largess given thee to give?
Profitless usurer, why dost thou use
So great a sum of sums, yet canst not live?
For having traffic with thyself alone,
Thou of thyself thy sweet self dost deceive:
Then how when nature calls thee to be gone,
What acceptable audit canst thou leave?
    Thy unused beauty must be tombed with thee,
    Which, used, lives th’ executor to be.

V

Those hours, that with gentle work did frame
The lovely gaze where every eye doth dwell,
Will play the tyrants to the very same
And that unfair which fairly doth excel;
For never-resting time leads summer on
To hideous winter, and confounds him there;
Sap checked with frost, and lusty leaves quite gone,
Beauty o’er-snowed and bareness every where:
Then were not summer’s distillation left,
A liquid prisoner pent in walls of glass,
Beauty’s effect with beauty were bereft,
Nor it, nor no remembrance what it was:
    But flowers distill’d, though they with winter meet,
    Leese but their show; their substance still lives sweet.

VI

Then let not winter’s ragged hand deface,
In thee thy summer, ere thou be distill’d:
Make sweet some vial; treasure thou some place
With beauty’s treasure ere it be self-kill’d.
That use is not forbidden usury,
Which happies those that pay the willing loan;
That’s for thyself to breed another thee,
Or ten times happier, be it ten for one;
Ten times thyself were happier than thou art,
If ten of thine ten times refigur’d thee:
Then what could death do if thou shouldst depart,
Leaving thee living in posterity?
    Be not self-will’d, for thou art much too fair
    To be death’s conquest and make worms thine heir.

VII

Lo! in the orient when the gracious light
Lifts up his burning head, each under eye
Doth homage to his new-appearing sight,
Serving with looks his sacred majesty;
And having climb’d the steep-up heavenly hill,
Resembling strong youth in his middle age,
Yet mortal looks adore his beauty still,
Attending on his golden pilgrimage:
But when from highmost pitch, with weary car,
Like feeble age, he reeleth from the day,
The eyes, ’fore duteous, now converted are
From his low tract, and look another way:
    So thou, thyself outgoing in thy noon:
    Unlook’d, on diest unless thou get a son.

VIII

Music to hear, why hear’st thou music sadly?
Sweets with sweets war not, joy delights in joy:
Why lov’st thou that which thou receiv’st not gladly,
Or else receiv’st with pleasure thine annoy?
If the true concord of well-tuned sounds,
By unions married, do offend thine ear,
They do but sweetly chide thee, who confounds
In singleness the parts that thou shouldst bear.
Mark how one string, sweet husband to another,
Strikes each in each by mutual ordering;
Resembling sire and child and happy mother,
Who, all in one, one pleasing note do sing:
    Whose speechless song being many, seeming one,
    Sings this to thee: ‘Thou single wilt prove none.’

IX

Is it for fear to wet a widow’s eye,
That thou consum’st thyself in single life?
Ah! if thou issueless shalt hap to die,
The world will wail thee like a makeless wife;
The world will be thy widow and still weep
That thou no form of thee hast left behind,
When every private widow well may keep
By children’s eyes, her husband’s shape in mind:
Look! what an unthrift in the world doth spend
Shifts but his place, for still the world enjoys it;
But beauty’s waste hath in the world an end,
And kept unused the user so destroys it.
    No love toward others in that bosom sits
    That on himself such murd’rous shame commits.

X

For shame! deny that thou bear’st love to any,
Who for thyself art so unprovident.
Grant, if thou wilt, thou art belov’d of many,
But that thou none lov’st is most evident:
For thou art so possess’d with murderous hate,
That ’gainst thyself thou stick’st not to conspire,
Seeking that beauteous roof to ruinate
Which to repair should be thy chief desire.
O! change thy thought, that I may change my mind:
Shall hate be fairer lodg’d than gentle love?
Be, as thy presence is, gracious and kind,
Or to thyself at least kind-hearted prove:
    Make thee another self for love of me,
    That beauty still may live in thine or thee.

XI

As fast as thou shalt wane, so fast thou grow’st,
In one of thine, from that which thou departest;
And that fresh blood which youngly thou bestow’st,
Thou mayst call thine when thou from youth convertest,
Herein lives wisdom, beauty, and increase;
Without this folly, age, and cold decay:
If all were minded so, the times should cease
And threescore year would make the world away.
Let those whom nature hath not made for store,
Harsh, featureless, and rude, barrenly perish:
Look, whom she best endow’d, she gave thee more;
Which bounteous gift thou shouldst in bounty cherish:
    She carv’d thee for her seal, and meant thereby,
    Thou shouldst print more, not let that copy die.

XII

When I do count the clock that tells the time,
And see the brave day sunk in hideous night;
When I behold the violet past prime,
And sable curls, all silvered o’er with white;
When lofty trees I see barren of leaves,
Which erst from heat did canopy the herd,
And summer’s green all girded up in sheaves,
Borne on the bier with white and bristly beard,
Then of thy beauty do I question make,
That thou among the wastes of time must go,
Since sweets and beauties do themselves forsake
And die as fast as they see others grow;
    And nothing ’gainst Time’s scythe can make defence
    Save breed, to brave him when he takes thee hence.

XIII

O! that you were your self; but, love you are
No longer yours, than you yourself here live:
Against this coming end you should prepare,
And your sweet semblance to some other give:
So should that beauty which you hold in lease
Find no determination; then you were
Yourself again, after yourself’s decease,
When your sweet issue your sweet form should bear.
Who lets so fair a house fall to decay,
Which husbandry in honour might uphold,
Against the stormy gusts of winter’s day
And barren rage of death’s eternal cold?
    O! none but unthrifts. Dear my love, you know,
    You had a father: let your son say so.

XIV

Not from the stars do I my judgement pluck;
And yet methinks I have astronomy,
But not to tell of good or evil luck,
Of plagues, of dearths, or seasons’ quality;
Nor can I fortune to brief minutes tell,
Pointing to each his thunder, rain and wind,
Or say with princes if it shall go well
By oft predict that I in heaven find:
But from thine eyes my knowledge I derive,
And constant stars in them I read such art
As ‘Truth and beauty shall together thrive,
If from thyself, to store thou wouldst convert’;
    Or else of thee this I prognosticate:
    ‘Thy end is truth’s and beauty’s doom and date.’

XV

When I consider everything that grows
Holds in perfection but a little moment,
That this huge stage presenteth nought but shows
Whereon the stars in secret influence comment;
When I perceive that men as plants increase,
Cheered and checked even by the self-same sky,
Vaunt in their youthful sap, at height decrease,
And wear their brave state out of memory;
Then the conceit of this inconstant stay
Sets you most rich in youth before my sight,
Where wasteful Time debateth with Decay
To change your day of youth to sullied night,
    And all in war with Time for love of you,
    As he takes from you, I engraft you new.

XVI

But wherefore do not you a mightier way
Make war upon this bloody tyrant, Time?
And fortify yourself in your decay
With means more blessed than my barren rhyme?
Now stand you on the top of happy hours,
And many maiden gardens, yet unset,
With virtuous wish would bear you living flowers,
Much liker than your painted counterfeit:
So should the lines of life that life repair,
Which this, Time’s pencil, or my pupil pen,
Neither in inward worth nor outward fair,
Can make you live yourself in eyes of men.
    To give away yourself, keeps yourself still,
    And you must live, drawn by your own sweet skill.

XVII

Who will believe my verse in time to come,
If it were fill’d with your most high deserts?
Though yet heaven knows it is but as a tomb
Which hides your life, and shows not half your parts.
If I could write the beauty of your eyes,
And in fresh numbers number all your graces,
The age to come would say ‘This poet lies;
Such heavenly touches ne’er touch’d earthly faces.’
So should my papers, yellow’d with their age,
Be scorn’d, like old men of less truth than tongue,
And your true rights be term’d a poet’s rage
And stretched metre of an antique song:
    But were some child of yours alive that time,
    You should live twice,—in it, and in my rhyme.

XVIII

Shall I compare thee to a summer’s day?
Thou art more lovely and more temperate:
Rough winds do shake the darling buds of May,
And summer’s lease hath all too short a date:
Sometime too hot the eye of heaven shines,
And often is his gold complexion dimm’d,
And every fair from fair sometime declines,
By chance, or nature’s changing course untrimm’d:
But thy eternal summer shall not fade,
Nor lose possession of that fair thou ow’st,
Nor shall death brag thou wander’st in his shade,
When in eternal lines to time thou grow’st,
    So long as men can breathe, or eyes can see,
    So long lives this, and this gives life to thee.

XIX

Devouring Time, blunt thou the lion’s paws,
And make the earth devour her own sweet brood;
Pluck the keen teeth from the fierce tiger’s jaws,
And burn the long-liv’d phoenix, in her blood;
Make glad and sorry seasons as thou fleets,
And do whate’er thou wilt, swift-footed Time,
To the wide world and all her fading sweets;
But I forbid thee one most heinous crime:
O! carve not with thy hours my love’s fair brow,
Nor draw no lines there with thine antique pen;
Him in thy course untainted do allow
For beauty’s pattern to succeeding men.
    Yet do thy worst, old Time; despite thy wrong,
    My love shall in my verse ever live young.

XX

A woman’s face with nature’s own hand painted,
Hast thou, the master mistress of my passion;
A woman’s gentle heart, but not acquainted
With shifting change, as is false women’s fashion:
An eye more bright than theirs, less false in rolling,
Gilding the object whereupon it gazeth;
A man in hue all ‘hues’ in his controlling,
Which steals men’s eyes and women’s souls amazeth.
And for a woman wert thou first created;
Till Nature, as she wrought thee, fell a-doting,
And by addition me of thee defeated,
By adding one thing to my purpose nothing.
    But since she prick’d thee out for women’s pleasure,
    Mine be thy love and thy love’s use their treasure.
"""

chars = sorted(list(set(sonnets)))
char_to_int = {c: i for i, c in enumerate(chars)}
int_to_char = {i: c for i, c in enumerate(chars)}
seq_length = 100
X = []
y = []

# Prepare the dataset of input to output pairs encoded as integers
for i in range(len(sonnets) - seq_length):
    X.append([char_to_int[c] for c in sonnets[i:i+seq_length]])
    y.append(char_to_int[sonnets[i+seq_length]])
X = np.array(X).reshape(len(X), seq_length, 1) / float(len(chars))
y = tf.keras.utils.to_categorical(y, num_classes=len(chars))

# Build the model
model_gen = tf.keras.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(len(chars), activation='softmax')
])

model_gen.build((None, seq_length, 1))
model_gen.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.01))
history_gen = model_gen.fit(X, y, epochs=20, batch_size=64)

Epoch 1/20
197/197 [==============================] - 6s 19ms/step - loss: 3.1528
Epoch 2/20
197/197 [==============================] - 4s 19ms/step - loss: 2.9035
Epoch 3/20
197/197 [==============================] - 4s 19ms/step - loss: 2.8066
Epoch 4/20
197/197 [==============================] - 4s 19ms/step - loss: 2.7615
Epoch 5/20
197/197 [==============================] - 4s 19ms/step - loss: 2.7103
Epoch 6/20
197/197 [==============================] - 4s 19ms/step - loss: 2.6862
Epoch 7/20
197/197 [==============================] - 4s 19ms/step - loss: 2.6631
Epoch 8/20
197/197 [==============================] - 4s 19ms/step - loss: 2.6328
Epoch 9/20
197/197 [==============================] - 4s 19ms/step - loss: 2.6183
Epoch 10/20
197/197 [==============================] - 4s 19ms/step - loss: 2.5896
Epoch 11/20
197/197 [==============================] - 4s 19ms/step - loss: 2.5741
Epoch 12/20
197/197 [==============================] - 4s 19ms/step - loss: 2.5570
Epoch 13/20
1

In [58]:
def generate_text(model, seed_text, n_chars=300, temperature=0.7):
    generated = seed_text
    for _ in range(n_chars):
        seq = generated[-seq_length:]
        seq_indices = [char_to_int.get(c, 0) for c in seq]
        if len(seq_indices) < seq_length:
            seq_indices = [0] * (seq_length - len(seq_indices)) + seq_indices
        x_pred = np.array(seq_indices).reshape(1, seq_length, 1) / float(len(chars))
        prediction = model.predict(x_pred, verbose=0)[0]
        prediction = np.log(prediction) / temperature
        exp_preds = np.exp(prediction)
        prediction = exp_preds / np.sum(exp_preds)
        next_index = np.random.choice(len(prediction), p=prediction)
        next_char = int_to_char[next_index]
        generated += next_char
    return generated

seed = "From fairest creatures we desire increase,"
print(generate_text(model_gen, seed))

From fairest creatures we desire increase, th  yeyiyi leceln po him.

XV

A 
 Soanligggsg,dnettinigs nnnel Taauty’s toan,
Tian oh the wilhes tfab mnte Fne mhk foa anoa 
And c aatrn’’eh men thy shi,
    Ant  dn thy ere mhre oopr tirt ane shne oaot,
    Thnss br the tnahl thine wht soll,goat’y tnev.
Ant heae uhe bein maio bhme ias brte?
Whic 
